In [1]:
%cd ..

d:\Projects\AI_projects\safety_helmet


## Import Libraries

In [2]:
import optuna
import yaml
import mlflow
import mlflow.pytorch
from ultralytics import YOLO

d:\Projects\AI_projects\safety_helmet\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read params file

In [3]:
with open(r"params.yaml", "r") as f:
    params = yaml.safe_load(f)

In [4]:
params

{'lr': {'min': 0.01, 'max': 0.1},
 'dp': {'min': 0.1, 'max': 0.6},
 'batch_options': [32, 64]}

In [5]:
params["lr"]["min"]

0.01

In [6]:
min_hp_lr = params["lr"]["min"]
max_hp_lr = params["lr"]["max"]


min_hp_dp = params["dp"]["min"]
max_hp_dp = params["dp"]["max"]

batch_options = params["batch_options"]

In [7]:
def goal_func(trial):
    
    lr = round(trial.suggest_float("lr", min_hp_lr, max_hp_lr),4)
    dropout = round(trial.suggest_float("dropout", min_hp_dp, max_hp_dp),2)
    batch = trial.suggest_categorical("batch", batch_options)
    

    model = YOLO("yolo11n.pt")

    model.train(
        data=r"safety-helmet-1\data.yaml",
        imgsz=128,
        epochs=100,
        project="runs",
        name=f"train_run_{trial.number}",
        
        lr0=lr,
        dropout=dropout,
        batch=batch,
        
        verbose=False
        )
    
    results = model.val(name=f"eval_run_{trial.number}")    
    map50 = results.box.map50
        
    with mlflow.start_run():
        mlflow.log_params({"lr": lr,"dropout": dropout,"batch": batch})
        mlflow.log_metric("map50", map50)
        model.export(format="onnx")
    return map50

mlflow.set_experiment("testing")
study = optuna.create_study(direction="maximize")
study.optimize(goal_func, n_trials=2)

print("Best trial:", study.best_trial)

2025/06/18 15:38:13 INFO mlflow.tracking.fluent: Experiment with name 'testing' does not exist. Creating a new experiment.
[I 2025-06-18 15:38:13,050] A new study created in memory with name: no-name-159ef559-20a1-4c90-8197-7115e9ead810


New https://pypi.org/project/ultralytics/8.3.156 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.155  Python-3.11.11 torch-2.7.1+cu126 CUDA:0 (Quadro P2000, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=safety-helmet-1\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.41, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=128, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0913, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_run_0, nbs=64, nms=False, opset=None, optim

train: Scanning D:\Projects\AI_projects\safety_helmet\safety-helmet-1\train\labels.cache... 330 images, 0 backgrounds, 5 corrupt: 100%|██████████| 335/335 [00:00<?, ?it/s]

train: D:\Projects\AI_projects\safety_helmet\safety-helmet-1\train\images\construction-concept-at-back-of-the-construction-worker-head-which-is-wearing-the-safety-helmet-and-checking-work-at-the-construction-site-non-existent-person-generative-ai-photo-1-_jpg.rf.3dbeb455151a256136e8ba29b6ce0c26.jpg: ignoring corrupt image/label: [Errno 2] No such file or directory: 'D:\\Projects\\AI_projects\\safety_helmet\\safety-helmet-1\\train\\images\\construction-concept-at-back-of-the-construction-worker-head-which-is-wearing-the-safety-helmet-and-checking-work-at-the-construction-site-non-existent-person-generative-ai-photo-1-_jpg.rf.3dbeb455151a256136e8ba29b6ce0c26.jpg'
train: D:\Projects\AI_projects\safety_helmet\safety-helmet-1\train\images\construction-concept-at-back-of-the-construction-worker-head-which-is-wearing-the-safety-helmet-and-checking-work-at-the-construction-site-non-existent-person-generative-ai-photo_jpg.rf.04e467241760f23b05e02e679cc6c590.jpg: ignoring corrupt image/label: [E

val: Fast image access  (ping: 0.10.0 ms, read: 57.412.9 MB/s, size: 11.5 KB)


...

                   all        117        445      0.908      0.924      0.957      0.619
Speed: 0.1ms preprocess, 3.7ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs\train_run_1
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'
Searching for 1 idle GPUs with free memory >= 20.0% and free utilization >= 0.0%...
Selected idle CUDA devices [0]
Ultralytics 8.3.155  Python-3.11.11 torch-2.7.1+cu126 CUDA:0 (Quadro P2000, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 119.9115.8 MB/s, size: 55.9 KB)


val: Scanning D:\Projects\AI_projects\safety_helmet\safety-helmet-1\valid\labels.cache... 117 images, 0 backgrounds, 0 corrupt: 100%|██████████| 117/117 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.54it/s]


                   all        117        445      0.951      0.886      0.956      0.621
Speed: 0.8ms preprocess, 4.9ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\eval_run_1
Ultralytics 8.3.155  Python-3.11.11 torch-2.7.1+cu126 CPU (Intel Core(TM) i7-8850H 2.60GHz)

PyTorch: starting from 'runs\train_run_1\weights\best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 6, 336) (5.2 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success  3.4s, saved as 'runs\train_run_1\weights\best.onnx' (9.9 MB)

Export complete (3.7s)
Results saved to D:\Projects\AI_projects\safety_helmet\runs\train_run_1\weights
Predict:         yolo predict task=detect model=runs\train_run_1\weights\best.onnx imgsz=128  
Validate:        yolo val task=detect model=runs\train_run_1\weights\best.onnx imgsz=128 data=safety-helmet-1\data.yaml  
Visualize:       https://netron.app


[I 2025-06-18 15:54:33,960] Trial 1 finished with value: 0.9564814152230414 and parameters: {'lr': 0.042794410923726496, 'dropout': 0.15924138207824454, 'batch': 32}. Best is trial 0 with value: 0.9564814152230414.


Best trial: FrozenTrial(number=0, state=1, values=[0.9564814152230414], datetime_start=datetime.datetime(2025, 6, 18, 15, 38, 13, 51150), datetime_complete=datetime.datetime(2025, 6, 18, 15, 45, 55, 896019), params={'lr': 0.09126034903592313, 'dropout': 0.40553187603217844, 'batch': 32}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'dropout': FloatDistribution(high=0.6, log=False, low=0.1, step=None), 'batch': CategoricalDistribution(choices=(32, 64))}, trial_id=0, value=None)
